In [1]:
import json
import random
import numpy as np
import pandas as pd
from csv import DictWriter
from deap import base, creator, tools

In [2]:
json_handle = open(r'../results/Customized_Data_2', 'r', encoding = 'utf-8')
json_string = json_handle.read()
instance = json.loads(json_string)
demand_ls = []
pop_ls = []
start_index = 1

for i in range(1, 101):
    demand = instance[f'customer_{i}']['demand']
    demand_ls.append(demand)

for demand in demand_ls:
    pop_ls.extend([start_index] * int(demand))
    start_index += 1

print(len(pop_ls))

1810


In [3]:
int_pop_ls = []
every_demand = []
int_start_index = 1
max_vehicle_number = instance['max_vehicle_number']

for demand in demand_ls:
    integer = demand // max_vehicle_number
    remainder = demand % max_vehicle_number
    
    if remainder == 0:
        int_pop_ls.extend([int_start_index] * int(integer))
        every_demand.extend([max_vehicle_number] * int(integer))
    else:
        int_pop_ls.extend([int_start_index] * (int(integer)+1))
        every_demand.extend([max_vehicle_number] * int(integer) + [int(remainder)])
    int_start_index += 1
    
integrate_customer_demand = list(zip(int_pop_ls, every_demand))   

#print(int_pop_ls)
print(integrate_customer_demand[:10])

[(1, 3), (1, 3), (1, 3), (1, 1), (2, 3), (2, 3), (2, 3), (2, 3), (2, 3), (2, 3)]


In [4]:
def ind2route(individual, instance): 
    '''decoding'''
    route = []
    depart_due_time = instance['depart']['due_time']
    max_vehicle_number = instance['max_vehicle_number']
    elapsed_time = 0
    initial_demand_ls = list(map(list, individual))

    for customer_demand in initial_demand_ls:
        
        customer_index = customer_demand[0]
        each_demand = customer_demand[1]
        
        due_time = instance[f'customer_{customer_index}']['due_time']
        service_time = instance[f'customer_{customer_index}']['service_time']
        travel_time = instance['distance_matrix'][0][customer_index]
            
        # Update elapsed time
        updated_elapsed_time = elapsed_time + each_demand*service_time + 2*travel_time

        # Validate vehicle load and elapsed time
        if each_demand !=0 and (updated_elapsed_time <= depart_due_time) and \
            (updated_elapsed_time - travel_time < due_time):   
            # Add to current sub-route
            route.append((customer_index, each_demand))
            elapsed_time = updated_elapsed_time
            each_demand = 0

    return route

In [5]:
def print_route(route): 
    '''print route'''
    route_str = '0'
    for customer_demand in route:
        customer_index = customer_demand[0]
        route_str = f'{route_str} - {customer_index} - 0'
    print(f"integrated_route: {route_str}")

In [6]:
def eval_vrptw(individual, instance, unit_cost=1.0, init_cost=0, wait_cost=0): 
    '''fitness function'''
    route = ind2route(individual, instance)
    total_unsaving_demand = len(pop_ls)
    max_vehicle_number = instance['max_vehicle_number']
    elapsed_time = 0
    total_cost = 0
    time_cost = 0
    unsaving_cost = 0
    
    for customer_demand in route:
        customer_index = customer_demand[0]
        each_demand = customer_demand[1]
        
        ready_time = instance[f'customer_{customer_index}']['ready_time']
        service_time = instance[f'customer_{customer_index}']['service_time']
        distance = instance['distance_matrix'][0][customer_index]
        
        # Calculate time cost
        arrival_time = elapsed_time + distance
        time_cost += max_vehicle_number * wait_cost * max(ready_time - arrival_time, 0)
        # Update elapsed time
        elapsed_time = arrival_time + each_demand*service_time + distance
        
        # Calculate unsaving demand
        total_unsaving_demand += - each_demand
        
    # Calculate unsaving cost
    unsaving_cost = init_cost + unit_cost * total_unsaving_demand
    
    # Obtain total cost
    total_cost = time_cost + unsaving_cost
    
    fitness = 1.0 / total_cost * 10**6
    return (fitness, )

In [7]:
def mut_inverse_indexes(individual): 
    '''mutation rules'''
    start, stop = sorted(random.sample(range(len(individual)), 2))
    temp = individual[start:stop+1]
    temp.reverse()
    individual[start:stop+1] = temp
    return (individual, )

In [8]:
def run_gavrptw(instance, unit_cost, init_cost, wait_cost, ind_size, pop_size, \
    cx_pb, mut_pb, n_gen, export_csv=False):
    
    
    #define types
    creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
    creator.create('Individual', list, fitness=creator.FitnessMax)

    #initialization
    toolbox = base.Toolbox()
    toolbox.register('indexes', random.sample, integrate_customer_demand, ind_size)
    toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.indexes)
    toolbox.register('population', tools.initRepeat, list, toolbox.individual)

    #operatiors
    toolbox.register('evaluate', eval_vrptw, instance=instance, unit_cost=unit_cost, \
                      init_cost=init_cost, wait_cost=wait_cost)
    toolbox.register('select', tools.selRoulette)
    toolbox.register('mate', tools.cxTwoPoint)
    toolbox.register('mutate', mut_inverse_indexes)
    
    pop = toolbox.population(n=pop_size)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(n_gen):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cx_pb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mut_pb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        print(f' Evaluated {len(invalid_ind)} individuals')
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum([x**2 for x in fits])
        std = abs(sum2 / length - mean**2)**0.5
        print(f'  Min {round(min(fits), 6)}')
        print(f'  Max {round(max(fits), 6)}')
        print(f'  Avg {round(mean, 6)}')
        print(f'  Std {round(std, 6)}')
        
    print('-- End of (successful) evolution --')
    
    best_ind = tools.selBest(pop, 1)[0]
    print(f'the best individual: {best_ind}')
    print(f'Fitness: {round(best_ind.fitness.values[0], 6)}')
    print_route(ind2route(best_ind, instance))
    print(f'Total cost: {1 / best_ind.fitness.values[0] * 10**6}')
   
        
    return pop

In [9]:
def main():
    '''main()'''
    random.seed(2022)

    instance = json.loads(json_string)

    unit_cost = 500.0
    init_cost = 0.0
    wait_cost = 10

    ind_size = 300
    pop_size = 100
    cx_pb = 0.9
    mut_pb = 0.1
    n_gen = 100

    export_csv = True

    run_gavrptw(instance=instance, unit_cost=unit_cost, init_cost=init_cost, \
        wait_cost=wait_cost, ind_size=ind_size, pop_size=pop_size, \
        cx_pb=cx_pb, mut_pb=mut_pb, n_gen=n_gen, export_csv=export_csv)


if __name__ == '__main__':
    main()

 Evaluated 83 individuals
  Min 1.02182
  Max 1.082929
  Avg 1.064009
  Std 0.011894
 Evaluated 90 individuals
  Min 1.040161
  Max 1.084186
  Avg 1.065282
  Std 0.009698
 Evaluated 81 individuals
  Min 1.040161
  Max 1.082536
  Avg 1.065791
  Std 0.009301
 Evaluated 88 individuals
  Min 1.042917
  Max 1.082536
  Avg 1.067346
  Std 0.009242
 Evaluated 95 individuals
  Min 1.047461
  Max 1.085808
  Avg 1.068835
  Std 0.00934
 Evaluated 89 individuals
  Min 1.049509
  Max 1.085808
  Avg 1.069739
  Std 0.009003
 Evaluated 94 individuals
  Min 1.052006
  Max 1.088607
  Avg 1.069837
  Std 0.008401
 Evaluated 89 individuals
  Min 1.035478
  Max 1.082929
  Avg 1.068087
  Std 0.009018
 Evaluated 96 individuals
  Min 1.052115
  Max 1.0813
  Avg 1.066626
  Std 0.009032
 Evaluated 89 individuals
  Min 1.052115
  Max 1.0813
  Avg 1.066141
  Std 0.008975
 Evaluated 93 individuals
  Min 1.046355
  Max 1.0839
  Avg 1.067972
  Std 0.008768
 Evaluated 85 individuals
  Min 1.052115
  Max 1.080424
  Avg 

 Evaluated 93 individuals
  Min 1.064643
  Max 1.079587
  Avg 1.073125
  Std 0.003733
 Evaluated 96 individuals
  Min 1.065516
  Max 1.079649
  Avg 1.072959
  Std 0.003827
 Evaluated 87 individuals
  Min 1.064934
  Max 1.080471
  Avg 1.072537
  Std 0.004177
-- End of (successful) evolution --
the best individual: [(37, 3), (88, 3), (37, 3), (42, 3), (36, 3), (65, 3), (7, 3), (56, 3), (35, 3), (39, 3), (95, 3), (5, 1), (94, 3), (28, 3), (16, 3), (60, 3), (57, 3), (50, 3), (93, 3), (61, 3), (87, 3), (93, 3), (24, 3), (18, 3), (20, 3), (7, 3), (46, 3), (61, 3), (15, 3), (96, 3), (2, 3), (42, 3), (87, 3), (68, 1), (61, 3), (78, 2), (1, 3), (50, 3), (16, 3), (12, 2), (74, 3), (6, 3), (81, 3), (63, 3), (71, 3), (8, 3), (95, 3), (63, 3), (85, 3), (71, 3), (62, 3), (49, 3), (54, 3), (90, 3), (56, 3), (79, 1), (95, 3), (81, 3), (85, 3), (48, 3), (20, 3), (2, 3), (22, 3), (21, 2), (12, 3), (56, 3), (59, 3), (89, 3), (47, 3), (82, 3), (61, 3), (8, 3), (34, 2), (59, 3), (92, 3), (58, 3), (84, 2), 